In [ ]:
import cv2
import pytesseract
import numpy as np
import re

img = cv2.imread("input.jpg")

# 拡大（重要）
img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 白い領域を抽出
_, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

# ノイズ除去
kernel = np.ones((5,5), np.uint8)
binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

# 輪郭検出
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

numbers = []

for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)

    # 小さすぎる領域は除外
    if w < 150 or h < 50:
        continue

    # 横長の矩形だけ使う（ラベル想定）
    if w / h < 2:
        continue

    roi = gray[y:y+h, x:x+w]

    # ROIだけ二値化
    _, roi_bin = cv2.threshold(roi, 180, 255, cv2.THRESH_BINARY_INV)

    config = r'--oem 1 --psm 7 -c tessedit_char_whitelist=0123456789.[]'
    text = pytesseract.image_to_string(roi_bin, config=config)

    match = re.findall(r'\d{2,3}\.\d{1,2}', text)
    if match:
        numbers.extend(match)

print("抽出結果:", numbers)
import time